In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [35]:
!pip install -q patchify

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [36]:
!rm -rf /kaggle/working/dset
!mkdir /kaggle/working/dset
!mkdir /kaggle/working/model
!cp -r /kaggle/input/blur-dataset/defocused_blurred /kaggle/input/blur-dataset/motion_blurred /kaggle/input/blur-dataset/sharp /kaggle/working/dset/


/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
mkdir: cannot create directory ‘/kaggle/working/model’: File exists
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [37]:
base_dir = "/kaggle/working/dset"
os.listdir(base_dir)


['motion_blurred', 'defocused_blurred', 'sharp']

In [38]:
!nvidia-smi

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Tue Feb 21 10:03:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    32W /  70W |  14454MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--------------

In [39]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

class ClassToken(Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls = tf.cast(cls, dtype=inputs.dtype)
        return cls

def mlp(x, cf):
    x = Dense(cf["mlp_dim"], activation="gelu")(x)
    x = Dropout(cf["dropout_rate"])(x)
    x = Dense(cf["hidden_dim"])(x)
    x = Dropout(cf["dropout_rate"])(x)
    return x

def transformer_encoder(x, cf):
    skip_1 = x
    x = LayerNormalization()(x)
    x = MultiHeadAttention(
        num_heads=cf["num_heads"], key_dim=cf["hidden_dim"]
    )(x, x)
    x = Add()([x, skip_1])

    skip_2 = x
    x = LayerNormalization()(x)
    x = mlp(x, cf)
    x = Add()([x, skip_2])

    return x

def ViT(cf):
    '''Input layer'''
    input_shape = (cf["num_patches"], cf["patch_size"]*cf["patch_size"]*cf["num_channels"])
    inputs = Input(shape=input_shape)
    # print(inputs.shape)

    """ Patch + Position Embeddings """
    patch_embed = Dense(cf["hidden_dim"])(inputs)   ## (None, 256, 768)
    # print(patch_embed.shape)

    positions = tf.range(start=0, limit=cf["num_patches"], delta=1)
    pos_embed = Embedding(input_dim=cf["num_patches"], output_dim=cf["hidden_dim"])(positions) ## (256, 768)
    embed = patch_embed + pos_embed ## (None, 256, 768)

    """ Adding Class Token """
    token = ClassToken()(embed)
    x = Concatenate(axis=1)([token, embed]) ## (None, 257, 768)

    for _ in range(cf["num_layers"]):
        x = transformer_encoder(x, cf)

    """ Classification Head """
    x = LayerNormalization()(x)     ## (None, 257, 768)
    x = x[:, 0, :]
    x = Dense(cf["num_classes"], activation="softmax")(x)

    model = Model(inputs, x)
    return model

In [40]:
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from patchify import patchify
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping

In [41]:
""" Hyperparameters """
hp = {}
hp["image_size"] = 200
hp["num_channels"] = 3
hp["patch_size"] = 25
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 16
hp["lr"] = 1e-4
hp["num_epochs"] = 500
hp["num_classes"] = 3
hp["class_names"] = ['F', 'M', 'S'] # F = defocused_blurred, M = Motion_blurred, S = Sharp

hp["num_layers"] = 12
hp["hidden_dim"] = 768
hp["mlp_dim"] = 3072
hp["num_heads"] = 12
hp["dropout_rate"] = 0.1

In [42]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path, split=0.1):

    #images extension is jpg and JPG
    valid_exts = [".jpg", ".jpeg"]

    # Recursively search for image files in all subdirectories under root_dir
    images = []
    for dirpath, dirnames, filenames in os.walk(path):
        for filename in filenames:
            # Check if the file has a valid image file extension
            if any(filename.lower().endswith(ext) for ext in valid_exts):
                # Append the file path to the list of image files
                images.append(os.path.join(dirpath, filename))

    np.random.shuffle(images)

    split_size = int(split*len(images))

    # print(split_size)
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)

    return train_x, valid_x, test_x

def process_image_label(path):
    """ Reading images """
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (hp["image_size"], hp["image_size"]))
    image = image / 255.0

    """ Preprocessing to patches """
    patch_shape = (hp["patch_size"], hp["patch_size"], hp["num_channels"])
    patches = patchify(image, patch_shape, hp["patch_size"])

    # Flatten the patches
    patches = patches.reshape(hp["flat_patches_shape"])
    # print(patches.shape)
    patches = patches.astype(np.float32)


    # Save the first 64 patches as images for debugging purposes
    # for i in range(64):
    #     cv2.imwrite("files/{}.jpg".format(i), patches[i])


    """ Label """
    class_name = path.split('_')[-1].split('.')[0]
    # print(class_name)
    class_index = hp["class_names"].index(class_name)
    class_index = np.array(class_index, dtype=np.int32)
    return patches, class_index
    
#to use the function in tensorlow we have to parse the functions
def parse(path):
    patches, labels = tf.numpy_function(process_image_label, [path], [tf.float32, tf.int32])
    labels = tf.one_hot(labels, hp["num_classes"])

    patches.set_shape([hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"]])
    labels.set_shape([hp["num_classes"]])

    return patches, labels

def tf_dataset(images, batch=32):
    dataset = tf.data.Dataset.from_tensor_slices(images)
    dataset = dataset.map(parse).batch(batch).prefetch(8)
    return dataset

In [43]:
if __name__ == '__main__':

    #seed the environment
    np.random.seed(42)
    tf.random.set_seed(42)

    create_dir("files")

    #paths to the dataset, model and history
    dset_path = '/kaggle/working/dset'
    model_path = os.path.join('/kaggle/working/model', 'model.h5')
    history_path = os.path.join('/kaggle/working/model', 'log.csv')

    #load the data
    train_x, valid_x, test_x = load_data(dset_path)
    print(f"Train: {len(train_x)} Valid: {len(valid_x)} Test: {len(test_x)}")

    # create the dataset
    train_dataset = tf_dataset(train_x, batch=hp["batch_size"])
    valid_dataset = tf_dataset(valid_x, batch=hp["batch_size"])

    #create the model
    """ Model """
    model = ViT(hp)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(hp["lr"], clipvalue=1.0),
        metrics=["acc"]
    )

    # as the kaggle provides two gpus, we can use both of them
    strategy = tf.distribute.MirroredStrategy()

    with strategy.scope():
        model = ViT(hp)
        model.compile(
            loss="categorical_crossentropy",
            optimizer=tf.keras.optimizers.Adam(hp["lr"], clipvalue=1.0),
            metrics=["acc"]
        )

        callbacks = [
            ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-10, verbose=1),
            CSVLogger(history_path, append=True),
            EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False),
        ]

    # train the model
    history = model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=hp["num_epochs"],
        callbacks=callbacks
    )

Train: 840 Valid: 105 Test: 105
Epoch 1/500
53/53 [==============================] - ETA: 0s - loss: 1.8034 - acc: 0.3393
Epoch 1: val_loss improved from inf to 1.18161, saving model to /kaggle/working/model/model.h5
53/53 [==============================] - 115s 2s/step - loss: 1.8034 - acc: 0.3393 - val_loss: 1.1816 - val_acc: 0.3238 - lr: 1.0000e-04
Epoch 2/500
53/53 [==============================] - ETA: 0s - loss: 1.2561 - acc: 0.3262
Epoch 2: val_loss improved from 1.18161 to 1.12775, saving model to /kaggle/working/model/model.h5
53/53 [==============================] - 86s 2s/step - loss: 1.2561 - acc: 0.3262 - val_loss: 1.1278 - val_acc: 0.3238 - lr: 1.0000e-04
Epoch 3/500
53/53 [==============================] - ETA: 0s - loss: 1.2020 - acc: 0.3214
Epoch 3: val_loss improved from 1.12775 to 1.12391, saving model to /kaggle/working/model/model.h5
53/53 [==============================] - 82s 2s/step - loss: 1.2020 - acc: 0.3214 - val_loss: 1.1239 - val_acc: 0.3238 - lr: 1.0000e

In [44]:
for device in tf.config.list_physical_devices():
    print(": {}".format(device.name))

: /physical_device:CPU:0
: /physical_device:GPU:0
: /physical_device:GPU:1
